In [5]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.io import DataLoader, TableNamesHOSP, TableNamesICU, ParquetConverter
import pandas as pd
import dask.dataframe as dd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=4, processes=True, memory_limit='8GB')
client

2025-05-15 23:31:58,288 - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2025-05-15 23:31:58,300 - INFO - State start
2025-05-15 23:31:58,300 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/worker-_1qatckd', purging
2025-05-15 23:31:58,301 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/worker-f7wepwv2', purging
2025-05-15 23:31:58,302 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/scheduler-i4ghlwue', purging
2025-05-15 23:31:58,304 - INFO -   Scheduler at:     tcp://127.0.0.1:54596
2025-05-15 23:31:58,305 - INFO -   dashboard at:  http://127.0.0.1:8787/status
2025-05-15 23:31:58,305 - INFO - Registering Worker plugin shuffle
2025-05-15 23:31:58,310 - INFO -         Start Nanny at: 'tcp://127.0.0.1:54599'
20

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54596,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:54603,Total threads: 4
Dashboard: http://127.0.0.1:54605/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:54599,


2025-05-15 23:33:57,551 - WARNING - Shuffle 6966330382159fd295a872b23c732def initialized by task ('shuffle-transfer-6966330382159fd295a872b23c732def', 0) executed on worker tcp://127.0.0.1:54603
2025-05-15 23:33:57,571 - WARNING - Shuffle 6966330382159fd295a872b23c732def deactivated due to stimulus 'task-finished-1747366437.570221'


## Scan the directory

In [8]:

data_loader = DataLoader()

# Scan the directory
data_loader.scan_mimic_directory()


data_loader.study_tables_info

,module,table_name,file_path,file_size,display_name,suffix,columns_list
6,hosp,diagnoses_icd,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,75013787,diagnoses_icd 75.0 MB,.parquet,"{seq_num, icd_code, hadm_id, icd_version, subj..."
9,hosp,patients,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,4582613,patients 4.6 MB,.parquet,"{dod, anchor_year_group, anchor_year, gender, ..."
13,hosp,poe,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,1054299614,poe 1.1 GB,.parquet,"{poe_id, transaction_type, order_type, orderti..."
14,hosp,admissions,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,25736908,admissions 25.7 MB,.parquet,"{language, dischtime, admission_location, race..."
15,hosp,d_icd_diagnoses,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,2683277,d_icd_diagnoses 2.7 MB,.parquet,"{long_title, icd_code, icd_version}"
20,hosp,poe_detail,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,105773003,poe_detail 105.8 MB,.parquet,"{poe_id, poe_seq, field_name, field_value, sub..."


In [15]:
%%time

subject_ids = data_loader.get_partial_subject_id_list_for_partial_loading(num_subjects=5)
print(subject_ids)

df = data_loader.load_table(table_name=TableNamesHOSP.DIAGNOSES_ICD, partial_loading=True, subject_ids=subject_ids)
df = df.compute()
df

2025-05-15 23:38:26,156 - INFO - Loaded diagnoses_icd full table. 9547064 rows.
2025-05-15 23:38:26,329 - INFO - Applied filters to diagnoses_icd table. 5179551 rows.
2025-05-15 23:38:26,330 - INFO - Filtering diagnoses_icd by subject_id for 5 subjects.


[10000032, 10000068, 10000084, 10000108, 10000117]


2025-05-15 23:38:27,995 - distributed.worker - ERROR - Compute Failed
Key:       ('getitem-aa18a941bfd27bb891b8ec81d581f861', 0)
State:     executing
Task:  <Task ('getitem-aa18a941bfd27bb891b8ec81d581f861', 0) getitem(...)>
Exception: "IndexError('boolean index did not match indexed array along axis 0; size of axis is 9547064 but size of corresponding boolean axis is 11426810')"
Traceback: '  File "/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/mimic_iv_code/.venv/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 5416, in __getitem__\n    result = getitem(key)\n             ^^^^^^^^^^^^\n'



IndexError: boolean index did not match indexed array along axis 0; size of axis is 9547064 but size of corresponding boolean axis is 11426810

In [14]:
tables_dict = data_loader.load_all_study_tables(partial_loading=True, num_subjects=5)


2025-05-15 23:37:35,456 - INFO - Loaded diagnoses_icd full table. 9547064 rows.
2025-05-15 23:37:35,661 - INFO - Applied filters to diagnoses_icd table. 5179551 rows.
2025-05-15 23:37:35,661 - INFO - Filtering diagnoses_icd by subject_id for 5 subjects.
2025-05-15 23:37:37,442 - distributed.worker - ERROR - Compute Failed
Key:       ('getitem-aa18a941bfd27bb891b8ec81d581f861', 0)
State:     executing
Task:  <Task ('getitem-aa18a941bfd27bb891b8ec81d581f861', 0) getitem(...)>
Exception: "IndexError('boolean index did not match indexed array along axis 0; size of axis is 9547064 but size of corresponding boolean axis is 11426810')"
Traceback: '  File "/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/mimic_iv_code/.venv/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 5416, in __getitem__\n    result = getitem(key)\n             ^^^^^^^^^^^^\n'



IndexError: boolean index did not match indexed array along axis 0; size of axis is 9547064 but size of corresponding boolean axis is 11426810

In [ ]:
print(tables_dict.keys())


patients_df        = tables_dict[TableNamesHOSP.PATIENTS.value]
admissions_df      = tables_dict[TableNamesHOSP.ADMISSIONS.value]
diagnoses_icd_df   = tables_dict[TableNamesHOSP.DIAGNOSES_ICD.value]
poe_df             = tables_dict[TableNamesHOSP.POE.value]
d_icd_diagnoses_df = tables_dict[TableNamesHOSP.D_ICD_DIAGNOSES.value]
poe_detail_df      = tables_dict[TableNamesHOSP.POE_DETAIL.value]

In [ ]:
df12 = patients_df.merge(admissions_df, on='subject_id', how='inner')
df34 = diagnoses_icd_df.merge(d_icd_diagnoses_df, on=('icd_code', 'icd_version'), how='inner')
df56 = poe_df.merge(poe_detail_df, on=('poe_id', 'poe_seq', 'subject_id'), how='left')

df1234   = df12.merge(df34, on=('subject_id', 'hadm_id'), how='inner')
df123456 = df1234.merge(df56, on=('subject_id', 'hadm_id'), how='inner')

In [ ]:
df123456c = df123456.compute()
df123456c

In [ ]:
# Filter age
df123456 = df123456[(df123456.anchor_age >= 18.0) & (df123456.anchor_age <= 75.0)]

# Filter icd_version. Ony keep the ICD 10
df123456 = df123456[df123456.icd_version == 10]

# remove columns discontinued_by_poe_id
df123456 = df123456.drop(columns=['discontinued_by_poe_id'])


In [ ]:
df = df123456.compute()
df.keys()

In [ ]:
# how to check if either one or both of admittime or dischtime are null
admissions_df_full    = data_loader.load_table(table_name=TableNamesHOSP.ADMISSIONS, partial_loading=False)
diagnoses_icd_df_full = data_loader.load_table(table_name=TableNamesHOSP.DIAGNOSES_ICD, partial_loading=False)

# filter out the admissions where patient dies
admissions_df_full = admissions_df_full[admissions_df_full.deathtime.isnull()]

# filter out the diagnoses where icd_version is not 10
diagnoses_icd_df_full = diagnoses_icd_df_full[diagnoses_icd_df_full.icd_version == 10]


dfa = admissions_df_full.compute()
dfd = diagnoses_icd_df_full.compute()

In [ ]:
diagnoses_icd_df_full.shape[0].compute()

In [ ]:
diagnoses_icd_df_full.size.compute()